# TPC2 - Física Estatística

# Atenção: 
* Não deve escrever o seu nome nem nenhum código identificador em nenhum sítio.
* O trabalho é individual. Podem e devem haver discussões com colegas mas o resultado entregue é individual. 
* Deve entregar ESTE Notebook de Jupyter
* Não deve acrescentar ou apagar nenhuma célula
* Todas as explicações devem ser claras e concisas.
* É preferível fazer menos e fazer bem que muito mal.
* O pacote numba pode diminuir o tempo de cálculo. 
* A não verificação de alguma destas regras leva ao anulamento e exclusão da prova.

In [12]:
# Introduza aqui funcoes gerais

import numpy as np
import numpy.random as rd
import random
import matplotlib.pyplot as plt


# Exercício 1
Nas aulas discutimos a solução do modelo de Ising de campo médio de
spin um ( $s_{i}=\pm1,0$ ) com $N$ spins. Neste trabalho de casa vamos
revisitá-lo numericamente. Considere o Hamiltoniano, 

\begin{align}
H & =\frac{1}{N}\sum_{i=1}^{N}\sum_{j\ne i}^{N}\left(1-s_{i}s_{j}\right)
\end{align}

onde a magnetização é dada por $M=\sum_{i=1}^{N}s_{i}$. Para este
modelo a energia é função apenas da magnetização e do número de partículas.
A densidade de estados do sistema é obtida usando o número de spins
$s_{i}=+1$ ($N_{+}$), o número de spins $s_{i}=-1$ ($N_{-}$) e
o número de spins $s_{i}=0$ ($N_{0}$): 
\begin{align*}
\Omega(N_{+},N_{-},N_{0}) & =\frac{N!}{N_{+}!N_{-}!N_{0}!} & N_{+} & =\frac{N-N_{0}+M}{2} & N_{-} & =\frac{N-N_{0}-M}{2}
\end{align*}
Com esta parametrização,
\begin{equation*}
\Omega(M,N_{0})=\frac{N!}{\left(\frac{N-N_{0}+M}{2}\right)!\left(\frac{N-N_{0}-M}{2}\right)!N_{0}!}
\end{equation*}
onde $M\in\{-(N-N_{0}),-(N-N_{0})+2,\dots,N-N_{0}-2,N-N_{0}\}$.

1 Calcule numericamente o valor médio exacto como função da temperatura
do módulo da magnetização para $\beta$ entre 0.1 e 1.5.
\begin{align*}
\left\langle \frac{|M|}{N}\right\rangle  & =\sum_{M,N_{0}}\frac{|M|}{N}\Omega(M,N_{0})\frac{e^{-\beta E(M)}}{Z(\beta)}\\
Z(\beta) & =\sum_{M,N_{0}}\Omega(M,N_{0})e^{-\beta E(M)}
\end{align*}
Sugestão: O cálculo da combinatórica envolve números muito grandes.
O ideia é usar a fórmula de Stirling,
\begin{equation*}
\log(n!)=n\log(n)-n+\frac{1}{2}\log(2\pi n)
\end{equation*}
e notar que o valor médio fica inalterado se deslocar a energia $-\beta E(M)\to-\beta E(M)-C$
nas exponenciais. Logo o ideal é calcular,
\begin{equation*}
\left\langle \frac{|M|}{N}\right\rangle =\frac{\sum_{M,N_{0}}\frac{|M|}{N}\Omega(M,N_{0})e^{-\beta E(M)-C}}{\sum_{M,N_{0}}\Omega(M,N_{0})e^{-\beta E(M)-C}}
\end{equation*}
onde $C$ é o máximo de $\ln\Omega(M,N_{0})-\beta E(M)$ para cada
temperatura.

Responda aqui

In [16]:
# Escreva aqui codigo

# To-Do:
# 1. Calcular Número de Estados
# 2. Calcular Energia
# 3. Calcular Função de Partição
# 4. Calcular Magentização Média


def StirlingApprox(n) -> float:
  return n * np.log(n) - n + 0.5 * np.log(2* np.pi * n )


def number_of_states(M:int,N_0:int) -> float:

  if (M > (N_0 - N)) and (M < (N - N_0)) and (N_0 > 1):

    N_p = (N - N_0 + M)/2
    N_m = (N - N_0 - M)/2

    Np_fact = StirlingApprox(N_p)
    Nm_fact = StirlingApprox(N_m)
    N0_fact = StirlingApprox(N_0)
    N_fact = StirlingApprox(N)

    return N_p, N_m, Np_fact, Nm_fact, N0_fact, N_fact, N_fact/(Np_fact * Nm_fact * N0_fact)

  else:
    return 0


def Energy(M,N)-> float:
  return  N - (M**2)/2 


def max_C(M_l:list, N_0_l:list, beta):
  C_list = []

  for m in M_l:
    for n0 in N_0_l:

      Omega = number_of_states(m,n0)[-1]
      E = Energy(m,N)
      C = np.log(Omega) - beta * E

      C_list.append(C)

  return float(max(C_list)), C_list


def Z_partition(N:int, N_0_l:list, M_l:list, beta:float, C:float) -> float:

  sum = 0 

  for m in M_l:
    for n0 in N_0_l:

      Omega = number_of_states(m,n0)[-1]
      E = Energy(m,N)
      exp = np.exp(-beta * E - C)

      sum += Omega * exp

  return sum


def avg_magnetization(N:int, N_0_l:list, M_l:list, beta:list) -> float:

  results = []
   
  for b in beta:

    sum = 0 
    C = max_C(M_l, N_0_l,b)[0]
    Z = Z_partition(N, N_0_l, M_l, b, C) 
 
    for m in M_l:
      for n in N_0_l:

        Omega = number_of_states(m,n)[-1]
        E = Energy(m,N)
        exp = np.exp(-b * E - C)

        sum += (abs(m)/N) * Omega * exp / Z

    results.append(sum)

  return sum


2. Para cada temperatura a distribuição de probabilidades da magnetização
é dada por,
\begin{equation}
P_{\beta}(M)=\frac{\sum_{N_{0}}e^{\ln(\Omega(M,N_{0}))-\beta E(M)}}{Z(\beta)}\label{eq:exacta}
\end{equation}
Gere uma amostra de valores aleatórios de $M$ com a distribução pretendida
usando o método directo. Com esta amostra $\{M_{1},M_{2},\dots,M_{K}\}$

    a) Represente o histograma da amostra aleatória gerada e compare com a distribuição teórica para $\beta\in \{0.1,0.25,0.5,0.6,0.7,0.75,0.8,0.9,1.0,1.1,1.2\}.$

Responda aqui

In [17]:
# Escreva aqui codigo

# To-Do
# 5. Calcular Probabilidade de Aceitação 
# 5.1 Calcular constante para maximizar Probabilidade de Aceitação
# 6. Gerar Amostra Aleatória (Método de VonNeumann)
# 7. Representar Histograma
# 7.1 Calcular distribuição teórica



def C_max_prob_accept(delta:float, N:int, M:int, N_0_l:list, beta:float, Z:float) -> float:

  """
  Cálculo da constante para maximizar a probabilidade de aceitação.
  """

  C_list = []
  sum = 0

  for n in N_0_l:

    Omega = number_of_states(M,n)[-1]
    E = Energy(M,N)
    exp = np.exp(-beta * E)

    sum += exp

  C = delta * (1/Z) * sum
  C_list.append(C)

  return 1/max(C_list), C_list


def prob_accept(delta:float, N:int, M_l:list, N_0_l:list, beta_:float) -> list:

  """
  Cálculo da probabilidade de aceitação.
  """

  prob_dist_m = []
  sum = 0 
  Z = Z_partition(N, N_0_l, M_l, beta_, C = 0.00)                 

  for m in M_l:
    for n in N_0_l:

      C_prob = C_max_prob_accept(delta, N, m, N_0_l, beta_, Z)[0]  

      Omega = number_of_states(m,n)[-1]
      E = Energy(m,N)
      exp = np.exp(-beta_ * E)
      sum += Omega * exp 

    p_accept = delta * (1/Z) * sum * C_prob
    prob_dist_m.append(p_accept)
             

  return prob_dist_m


def teor_prob_dist(delta:float, N:int, M_l:list, N_0_l:list, beta:float) -> list:

  """
  Cálculo da distribuição teórica.
  """
  
  teor_prob_dist_m = []
  sum = 0 
  Z = Z_partition(N, N_0_l, M_l, beta, C = 0) 

  for m in M_l:
    for n in N_0_l:

      Omega = number_of_states(m,n)[-1]
      E = Energy(m,N)
      exp = np.exp(-beta * E)
      sum += Omega * exp 

    prob = sum / Z
    teor_prob_dist_m.append(prob)

  return teor_prob_dist_m



def distribution_gen(K:int, delta:float, beta: float, N_0_l:list) -> list:                              

    final = np.array([])

    while(final.size < K):

        M_l = random.sample(range(-delta, delta), K)
               
        results, = np.where(rd.random_sample(K) < prob_accept(delta, N, M_l, N_0_l, beta))

        results = M_l[results,]                                      

        final = np.concatenate((final,results))            

    return final[0:K]






# Representação do histograma 
N = 30

N_0_l = [2,2,2,2,2]
delta = 1000
K = 200
beta = [0.1, 0.25, 0.5, 0.6, 0.7, 0.75, 0.8, 0.9, 1.0, 1.1, 1.2]

for b in beta:

  bins = plt.hist(distribution_gen(K, delta, b, N_0_l),bins=200, density=True, stacked=True)[1]

  plt.plot(bins, teor_prob_dist(delta, N, bins, N_0_l, b), label = f'\beta = {b}')
  plt.legend(frameon=False,fontsize=10)
  plt.xlabel("M")
  plt.ylabel("$P_{\beta}$(M)")
  plt.show()


TypeError: ignored

(b) Meça a autocorrelação da magnetização para $\beta=0.8$,
\begin{align}
corr_{M}(\tau)	& =\frac{\left\langle M_{i}M_{i+\tau}\right\rangle -\left\langle M_{i}\right\rangle ^{2}}{\left\langle M_{i}^{2}\right\rangle -\left\langle M_{i}\right\rangle ^{2}} \\
\left\langle M_{i}M_{i+\tau}\right\rangle 	& =\frac{1}{L}\sum_{i=1}^{L}M_{i}M_{i+\tau}
\end{align}


Responda aqui

In [ ]:
# Escreva aqui codigo

# Exercício 2
Na pergunta anterior efectuamos uma simulação Monte Carlo num espaço
de fase a uma dimensão, da variável $M$. Será que esta dinâmica corresponde
à mesma dinâmica do sistema de $N$ spins com o Hamiltoniano 
\begin{equation*}
H=\frac{1}{N}\sum_{i=1}^{N}\sum_{j\ne i}^{N}\left(1-s_{i}s_{j}\right)
\end{equation*}
de campo médio e o algoritmo de Metropolis?
1 Para testar implemente o algoritmo de Metropolis para $N$ spins:
-  Atribua a cada um dos $N$ spins do sistema uma variável $s_{i}=\pm1,0$
com probabilidade uniforme.
- Considere um passo de tempo o seguinte algoritmo:
 - Escolha um spin com probabilidade uniforme.
 - Aceite trocar o seu spin com a probabilidade $\min\left(1,e^{-\beta\Delta E}\right)$
senão fique no mesmo estado.

Responda aqui

In [ ]:
# Escreva aqui codigo

2. Verifique se a distribuição gerada neste algoritmo é equivalente à descrita no problema anterior. Gerando a posição inicial a partir do densidade de estados (todos os estados são equiprovaveis) verifique que a distribuição de probabilidade ao final de um tempo fixo (t=N,2N,4N,16N) é igual em ambos os algoritmos.


Responda aqui

In [ ]:
# Escreva aqui codigo

3. Calcule a função de autocorrelação da energia no tempo de Monte Carlo. 

 (#) Pode  Escrever aqui texto 

In [ ]:
# Escreva aqui codigo

4. Se em vez de considerar o modelo de Ising de campo médio considerasse o de duas dimensões como seria o algoritmo? 

Responda aqui

In [ ]:
# Escreva aqui codigo

Bom trabalho